In [3]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [917 kB]
Hit:13 http://ppa.launchpad.net/

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-03 22:40:40--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-02-03 22:40:42 (1.22 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Final-Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
# Read in data from S3 Buckets for User Data
from pyspark import SparkFiles
url ="https://mthiobane-finalproject.s3.amazonaws.com/owid-covid-data.csv"
spark.sparkContext.addFile(url)
covid_data_df = spark.read.csv(SparkFiles.get("owid-covid-data.csv"), sep=",", header=True)

In [7]:
covid_data_df.show()

+--------+---------+-----------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+-----------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+--------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+--------------------------+-------------------------------------+------------------------------+-------------------------------

In [8]:
### We noticed that there are some rows where the continent and the location are the same which deters us from
### analyzing the correct data. ISO_codes with OWID abbreviations have the data of the entire continent so we cannot use
### that

### So what we do is clean the data. 

countries_df = covid_data_df[["iso_code", "continent", "location", "date", "new_cases", "new_deaths", "total_cases", "total_deaths", "reproduction_rate", "hosp_patients", "new_tests", "positive_rate", "new_vaccinations", "population", "median_age", "hospital_beds_per_thousand" ]]
countries_df.show()

+--------+---------+-----------+----------+---------+----------+-----------+------------+-----------------+-------------+---------+-------------+----------------+----------+----------+--------------------------+
|iso_code|continent|   location|      date|new_cases|new_deaths|total_cases|total_deaths|reproduction_rate|hosp_patients|new_tests|positive_rate|new_vaccinations|population|median_age|hospital_beds_per_thousand|
+--------+---------+-----------+----------+---------+----------+-----------+------------+-----------------+-------------+---------+-------------+----------------+----------+----------+--------------------------+
|     AFG|     Asia|Afghanistan|2020-02-24|      5.0|      null|        5.0|        null|             null|         null|     null|         null|            null|39835428.0|      18.6|                       0.5|
|     AFG|     Asia|Afghanistan|2020-02-25|      0.0|      null|        5.0|        null|             null|         null|     null|         null|       

In [9]:
### Selection the continent we want to Analyse (America is divided in North America and South America)
continents = ["Africa", "Asia", "Europe","North America", "Oceania", "South America"]
continents_df = countries_df[countries_df['location'].isin(continents)]
continents_df.show()

+--------+---------+--------+----------+---------+----------+-----------+------------+-----------------+-------------+---------+-------------+----------------+------------+----------+--------------------------+
|iso_code|continent|location|      date|new_cases|new_deaths|total_cases|total_deaths|reproduction_rate|hosp_patients|new_tests|positive_rate|new_vaccinations|  population|median_age|hospital_beds_per_thousand|
+--------+---------+--------+----------+---------+----------+-----------+------------+-----------------+-------------+---------+-------------+----------------+------------+----------+--------------------------+
|OWID_AFR|     null|  Africa|2020-02-13|      0.0|       0.0|       null|        null|             null|         null|     null|         null|            null|1373486472.0|      null|                      null|
|OWID_AFR|     null|  Africa|2020-02-14|      1.0|       0.0|        1.0|        null|             null|         null|     null|         null|            nu

In [10]:
import pandas as pd
import numpy as np

In [11]:
# Store environmental variable
from getpass import getpass
password = 'FinalProject'
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://finalproject.ccioeicmqb2h.us-east-1.rds.amazonaws.com:5432/finalproject"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

In [12]:
#Based on the Machine learning we divied the dataset in 3 groupes depending on the number of case :
# low, medium and High
# we randomly selectec 6 countries as below 
# Low : Cyprus, Dominican Republic
# med : jersey, Gambia
# high : Palestine, Oman
### Selection the continent we want to Analyse (America is divided in North America and South America)
countries_hml = ["Cyprus", "Dominican Republic", "jersey", "Gambia", "Palestine", "Oman"]
countries_hml = countries_df[countries_df['location'].isin(countries_hml)]
countries_hml.show()

+--------+---------+--------+----------+---------+----------+-----------+------------+-----------------+-------------+---------+-------------+----------------+----------+----------+--------------------------+
|iso_code|continent|location|      date|new_cases|new_deaths|total_cases|total_deaths|reproduction_rate|hosp_patients|new_tests|positive_rate|new_vaccinations|population|median_age|hospital_beds_per_thousand|
+--------+---------+--------+----------+---------+----------+-----------+------------+-----------------+-------------+---------+-------------+----------------+----------+----------+--------------------------+
|     CYP|   Europe|  Cyprus|2020-03-08|     null|      null|       null|        null|             null|         null|     null|         null|            null|  896005.0|      37.3|                       3.4|
|     CYP|   Europe|  Cyprus|2020-03-09|     null|      null|       null|        null|             null|          0.0|     null|         null|            null|  896

In [13]:
# Write DataFrame to active_user table in RDS
continents_df.write.jdbc(url=jdbc_url, table='Dataset_continents_data', mode=mode, properties=config)

In [14]:
# Write DataFrame to active_user table in RDS
countries_hml.write.jdbc(url=jdbc_url, table='Dataset_countries_hml_data', mode=mode, properties=config)